In [21]:
#BIbliotecas
import geopandas as gpd
import folium
from folium.plugins import HeatMap

In [22]:
#Ler Shapefile em um GeoDataframe
blocos = gpd.read_file('data/blocos_2020.shp')

blocos.head()

,id,nome,dia,hora,geometry
0,1,Mulheres de Carnaval,2020-02-15,11:00,POINT (-5329902.545 -1780292.187)
1,2,Essa Boquinha Eu Já Beijei,2020-02-15,11:00,POINT (-5331800.090 -1780302.408)
2,3,Nana Banana Bloquinho,2020-02-15,13:00,POINT (-5352776.499 -1791842.800)
3,4,Galo Cego,2020-02-15,14:00,POINT (-5330185.734 -1781429.150)
4,5,Pauta na Rua,2020-02-15,16:00,POINT (-5333231.967 -1781412.917)


In [23]:
#Modificando DATUM (WGS 84)
blocos = blocos.to_crs(epsg=4326)

#Extraindo coordenadas do campo "geometry"
blocos["x"] = blocos["geometry"].apply(lambda geom: geom.x)
blocos["y"] = blocos["geometry"].apply(lambda geom: geom.y)

#Criando uma lista com as coordenadas x e y
loc = list(zip(blocos["y"], blocos["x"]))
print(loc[:4])

[(-15.788927089722067, -47.87932919019098), (-15.789015444473506, -47.896375126521214), (-15.888748531682253, -48.084809418881335), (-15.798755020233463, -47.88187311795138)]


In [24]:
#Mapa estático

#Armazenando a geometria como um GeoJson
pts_gjson = folium.features.GeoJson(blocos, name="Blocos de carnaval 2020")

#Criando mapa
m = folium.Map(location=[-15.788497,-47.879873],  tiles = "CartoDB dark_matter", zoom_start=10, control_scale=True)

# parametros: HeatMap(data, name=None, min_opacity=0.5, max_zoom=18, max_val=1.0, radius=25, blur=15, gradient=None, overlay=True, control=True, show=True)
HeatMap(loc).add_to(m) #Adicionando mapa de calor

pts_gjson.add_to(m) #Adicionando pontos

m #Mostrar mapa na tela

In [25]:
#Mapa dinâmico (Interativo)

import plotly.graph_objects as go

#Token para o mapbox
mapbox_access_token = open("data/tk.mapbox_token").read()

#Definição dos campos de coordenadas e atributos que serão mostrados na janela pop-up
site_lat = blocos.y
site_lon = blocos.x
name = blocos['nome']
date = blocos['dia']
hour = blocos['hora']

#Criação do mapa
fig = go.Figure()

#Mapa de pontos
fig.add_trace(go.Scattermapbox(
        lat=site_lat,
        lon=site_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=4,
            color='rgb(0, 0, 0)',
            opacity=0.9
        ),
        hovertext= name + "\t " + date+ "\t "  + hour, #Texto da janela pop-up
        hoverinfo= 'text'
    ))

#Mapa de calor
fig.add_trace(go.Densitymapbox(lat=site_lat, lon=site_lon, 
                                 radius=15, hoverinfo='skip', showscale=False))

#Atributos do mapa em geral
fig.update_layout(
    title='Blocos de carnaval DF-2020',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(lat=-15.788497, lon=-47.879873),
        pitch=0,
        zoom=9,
        style='dark'
    ),
)

fig.show()